In [2]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as nprand

# Street Racer

In this notebook, you'll apply the methods of chapter 4 of Sutton's book to a simple racing problem.

The problem consists in driving a car as fast of possible over an exact distance $L$, and stopping there.

This distance is divided in steps $0, ..., L$. The car can drive at three different speed: _low_, _medium_, _high_. Leaving step $j$ at _low_ speed, it will move to $j+1$. _Medium_ and _high_ bring it to $j+2$ and $j+3$, respectively.

At any step, the driver can decide to _decelerate_, _maintain speed_ or _accelerate_. Decelarating will cause the car to leave its current place at one speed lower. If the car is already at _low_ speed, decelarating keeps it in the same spot. Maintaining speed does exactly what you think. Accelerating will increase the speed by one, except at _high_ speed, where it is equivalent to maintaining speed.

The car starts on step $0$ at _low_ speed.

Beyond the $L$ distance there is a huge, hot lake of lava. Needless to say, the car must be able to stop at $L$, or the driver will suffer quite a lot.

To help the driver win the race and not die, build a model of the problem and apply the policy iteration and value iteration methods to find her optimal trajectory.

As this problem is an (over-)simplification of our traffic light problem, any work done here could serve as a building block for later.

# Building the model

Start by figuring out the number of states you will need and build transition matrices for every action. For now, actions move the car from state to state in the deterministic manner described above.

In [86]:
l = 20

In [87]:
T_decelerate = np.zeros(shape=(3*l+1, 3*l+1))
T_maintain = np.zeros(shape=(3*l+1, 3*l+1))
T_accel = np.zeros(shape=(3*l+1, 3*l+1))

we have three states for each position then we have a lava state (absorbing state)

the matrix repeats itself for each action :  (here the decelerate matrix)

\begin{pmatrix}1&0&0 & 0&0&0 & 0&0&0 &  0&0&0 & \ldots 
\\ 0&0&0 & 1&0&0 & 0&0&0 &  0&0&0 & \ldots 
\\ 0&0&0 & 0&0&0 & 0&1&0 &  0&0&0 & \ldots
\\ 0&0&0 & 1&0&0 & 0&0&0 &  0&0&0 & \ldots
\\ 0&0&0 & 0&0&0 & 1&0&0 &  0&0&0 & \ldots
\\ 0&0&0 & 0&0&0 & 0&0&0 &  0&1&0 & \ldots
\end{pmatrix}

In [88]:
n=len(T_decelerate)

for i in range(n-1):
    if (i%3==0 ):
        T_decelerate[i][i]=1        #if at low speed and decelerating : stay on the same spot at low speed
    if (i%3==1):                
        if (i+2>n-1):
            T_decelerate[i][n-1]=1  #if action takes us beyond l : we fall into the lava-state  
        else :     
            T_decelerate[i][i+2]=1  #other wise we just end up in the next position with low speed since we are in medium 
    if (i%3==2):
        if (i+5>n-1):
            T_decelerate[i][n-1]=1  #if action takes us beyond l : we fall into the lava-state 
        else : 
            T_decelerate[i][i+5]=1  #otherwise we just end up two states after with medium speed 
                       

for i in range(n-1):
    if (i%3==0):
        if (i+7>n-1):
            T_accel[i][n-1]=1
        else :
            T_accel[i][i+7]=1
        if (i+3 > n-1 ):
            T_maintain[i][n-1]=1
        else:
            T_maintain[i][i+3]=1
    if (i%3==1):
        if (i+10>n-1):
            T_accel[i][n-1]=1
        else : 
            T_accel[i][i+10]=1
        if (i+6>n-1):
            T_maintain[i][n-1]=1
        else :
            T_maintain[i][i+6]=1
    if (i%3==2):
        if(i+9>n-1):
            T_accel[i][n-1]=1
            T_maintain[i][n-1]=1
        else :
            T_accel[i][i+9]=1
            T_maintain[i][i+9]=1

            
#once dead you stay dead !            
T_accel[n-1][n-1]=1
T_maintain[n-1][n-1]=1
T_decelerate[n-1][n-1]=1 

we check for any encoding issues in the matrices : 

In [89]:
def error_matrix(A):
    error=np.zeros(len(A))                  #we haven't detected any outcome to this action : issue
    for i in range(0,len(A)):
        for j in range(0,len(A)):
            if A[i][j]==1 and error[i]==0 : #we have one resulting state after this action, this is good
                error[i]=1
            elif A[i][j]==1 and error[i]==1: #we have more than one outcome : here issue since it's deterministic
                error[i]=2
    print("erreur de type pas d'actions associées ",np.where(error == 0)[0])
    print("erreur de type deux actions ou plus associées ",np.where(error == 2)[0],":")
    for i in np.where(error == 2)[0]:
        print(A[i])

In [90]:
print("deceleration matrix")
error_matrix(T_decelerate)
print("acceleration matrix")
error_matrix(T_accel)
print("maintain matrix")
error_matrix(T_maintain)

deceleration matrix
erreur de type pas d'actions associées  []
erreur de type deux actions ou plus associées  [] :
acceleration matrix
erreur de type pas d'actions associées  []
erreur de type deux actions ou plus associées  [] :
maintain matrix
erreur de type pas d'actions associées  []
erreur de type deux actions ou plus associées  [] :


And define the reward function

In [151]:
R = -np.ones(3*l+1)
lava = -10^l*100
R[-1]=  lava  #lava state : you die
R[-4]=10^l         #L in low speed ! win ! 

#are these ones really usefull ? bad reward when entering L state with medium or high speed
R[-2]= lava
R[-3]= lava

defining a policy : at the beginning we choose a random policy. A policy is here encoded as a sequence of letters : 1,2,3 for decelerate, maintain, accelerate. The sequence is of length 3*l+1

In [97]:
policy_initial = np.zeros(3*l+1)
for i in range(len(policy_initial)):
    policy_initial[i]=nprand.choice([1,2,3])

In [98]:
policy_initial

array([ 1.,  2.,  1.,  2.,  3.,  1.,  2.,  1.,  1.,  3.,  1.,  1.,  1.,
        2.,  3.,  2.,  2.,  3.,  2.,  1.,  1.,  3.,  2.,  2.,  3.,  3.,
        2.,  3.,  2.,  2.,  3.,  2.,  1.,  3.,  2.,  1.,  2.,  1.,  2.,
        1.,  1.,  1.,  2.,  3.,  3.,  1.,  2.,  2.,  3.,  3.,  1.,  3.,
        2.,  3.,  2.,  1.,  2.,  1.,  1.,  3.,  3.])

Here we define the find_new_state function, which takes as argument a policy p and an index i. The index i represents the state we are in actually and the function return in which state we will be if we follow the policy 

In [103]:
def find_new_state(p,i):

# we find the action the policy tells us to take then we scan the corresponding matrix to find the resulting state
# if the state is a "low speed state" and we choose to decelerate then we stay where we are, hence return i
    
    k=p[i]
    
    if k==1:
        if i%3 == 0:
            return i
        else :
            for j in range(len(T_decelerate[i])):
                if  j != i and T_decelerate[i][j]==1:
                    return  j    
    elif k ==2 :
        for j in range(len(T_maintain[i])):
            if  j != i and T_maintain[i][j]==1:
                return j
            return j
                
    elif k==3:
        for j in range(len(T_accel[i])):
            if  j != i and T_accel[i][j]==1:
                return j
            return j

    else :
         raise ValueError("k should be between 1 and 3")

Problème de modélisation de l'environnement : la voiture a techniquement la possibilité de "dépasser" L, j'ai pas encore trouvé de bonne façon d'empêcher ça... Une bonnne façon peut être est juste de modifier la fonction find new state d'une façon plus optimale (test i-i%3 --> position puis check vitesse puis check action pour trouve position future et ensuite se débrouiller)

Ou sinon rajouter un autre état, celui dans lequel il ne faudrait surtout pas aller et laisser tout le monde à -1, sauf L à +1 et lui à -10*3*l ? 

Tests while modeling everything : 

# Policy iteration

Apply the policy iteration procedure to figure out the best policy to follow.

In [129]:
#definition of  stopping criterion 
epsilon = 0.01
#definition of discount factor
gamma=0.9
#definitions of the states

this policy evaluation is only working for non-stochastic policies and is an in-place one 

In [154]:
lava

-2010

In [167]:
def policy_eval(p,epsilon):
    delta = 10
    V=np.zeros(3*l+1)
    bla = 0
    while(delta > epsilon):
        delta = 0
        for i in range(3*l+1):
            v=V[i]
            j=find_new_state(p,i)
            V[i]=R[j]+gamma*V[j]
            delta = max(delta,abs(v-V[i]))
        bla = bla+1
        print(delta)
        print(bla)
        
    return V

In [91]:
#print(find_new_state(policy_initial,52))
#policy_initial[52]

In [168]:
print([int(elem) for elem in policy_eval(policy_initial,0.001)])

2010.0
1
27.0
2
24.3
3
21.87
4
19.683
5
17.7147
6
15.94323
7
14.348907
8
12.9140163
9
11.62261467
10
10.460353203
11
9.4143178827
12
8.47288609443
13
7.62559748499
14
6.86303773649
15
6.17673396284
16
5.55906056656
17
5.0031545099
18
4.50283905891
19
4.05255515302
20
3.64729963772
21
3.28256967395
22
2.95431270655
23
2.6588814359
24
2.39299329231
25
2.15369396308
26
1.93832456677
27
1.74449211009
28
1.57004289908
29
1.41303860917
30
1.27173474826
31
1.14456127343
32
1.03010514609
33
0.927094631479
34
0.834385168331
35
0.750946651498
36
0.675851986348
37
0.608266787713
38
0.547440108942
39
0.492696098048
40
0.443426488243
41
0.399083839419
42
0.359175455477
43
0.323257909929
44
0.290932118936
45
0.261838907043
46
0.235655016338
47
0.212089514705
48
0.190880563234
49
0.171792506911
50
0.15461325622
51
0.139151930598
52
0.125236737538
53
0.112713063784
54
0.101441757406
55
0.0912975816651
56
0.0821678234986
57
0.0739510411487
58
0.0665559370339
59
0.0599003433305
60
0.0539103089974
61
0.0

 <font color='red'>
policy_improv is to be improved, but I don't know the exact command to do what I want, so it's a beginning 

In [132]:
def policy_improv(p,V,boolean):
    new_p=np.zeros(shape=3*l)
    
    for i in range(3*l):
        for j in range(len(T_maintain[i])):
            if  j != i and T_maintain[i][j]==1:
                k=j
        action_1=R[k]+gamma*V[k]
        for j in range(len(T_accel[i])):
            if  j != i and T_accel[i][j]==1:
                k=j
        action_2=R[k]+gamma*V[k]
        for j in range(len(T_decelerate[i])):
            if T_decelerate[i][j]==1:
                k=j
        action_3=R[k]+gamma*V[k]
        new_p[i]=np.argmax([action_1,action_2,action_3])+1
        if new_p[i]!=p[i]:
            boolean = False
    return new_p


Now combine the two functions to iterate over policies!

In [133]:
# policy iteration
p=policy_initial
Delta_iter=1
epsilon_iter=0.1
policy_stable=False

while  not policy_stable :
    V=policy_eval(p,epsilon)
    policy_stable= True
    p=policy_improv(p,V,policy_stable)
    


In [134]:
V

array([   -9.99700309,    -9.99730278,    -9.99730278,    -9.99730278,
          -9.99730278,    -9.99700309,    -9.99730278,    -9.99730278,
          -9.99730278,    -9.99730278,    -9.99700309,    -9.99730278,
          -9.99700309,    -9.99730278,    -9.99730278,    -9.99730278,
          -9.99730278,    -9.99730278,    -9.99730278,    -9.99730278,
          -9.99730278,    -9.99730278,    -9.99730278,    -9.99730278,
          -9.99730278,    -9.99730278,    -9.99730278,    -9.99730278,
          -9.99730278,    -9.99730278,    -9.99730278,    -9.99730278,
          -9.99700309,    -9.99730278,    -9.99730278,    -9.99730278,
          -9.99730278,    -9.99700309,    -9.99730278,    -9.99700309,
          -9.99730278,    -9.99730278,    -9.99730278,    -9.99730278,
          -9.99730278,    -9.99700309,    -9.99730278,    -9.99730278,
          -9.99730278,    -9.99730278,   268.9100928 ,    -9.99730278,
          -9.99730278,    -9.99730278,    -9.99730278,   299.9100928 ,
      

In [135]:
p

array([ 3.,  1.,  1.,  2.,  1.,  3.,  1.,  1.,  1.,  1.,  3.,  1.,  3.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  2.,  1.,  3.,  1.,  1.,  1.,  1.,  3.,  1.,
        3.,  2.,  1.,  1.,  3.,  1.,  3.,  1.,  1.,  2.,  1.,  3.,  1.,
        3.,  3.,  1.,  3.,  1.,  3.,  1.,  1.])

In [136]:
policy_initial

array([ 1.,  2.,  1.,  2.,  3.,  1.,  2.,  1.,  1.,  3.,  1.,  1.,  1.,
        2.,  3.,  2.,  2.,  3.,  2.,  1.,  1.,  3.,  2.,  2.,  3.,  3.,
        2.,  3.,  2.,  2.,  3.,  2.,  1.,  3.,  2.,  1.,  2.,  1.,  2.,
        1.,  1.,  1.,  2.,  3.,  3.,  1.,  2.,  2.,  3.,  3.,  1.,  3.,
        2.,  3.,  2.,  1.,  2.,  1.,  1.,  3.,  3.])

To figure out if everything is going well, make sure that at each iteration you keep track of the value vector, as well as the trajectory of the car according to the current policy. The latter allows you to compute the current policy's total reward and plot the evolution.

Then use the stored values to make a video similar to _street_racer.mp4_ on the repo. The following procedure can be used to save figures.

In [96]:
for idx, v in enumerate(values):
    v = np.array(v[:trap]).reshape(3, l)
    fig = plt.figure(figsize=(l*2, 6), dpi=72)
    ax = fig.add_subplot(111)
    ax.imshow(v, interpolation='nearest', cmap='gray')
    plt.yticks([])
    plt.savefig('img/value_'+str(idx)+'.jpg', dpi=72, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

NameError: name 'values' is not defined

Install the command-line utility _ffmpeg_ and use it to transform the saved sequence of images into a mp4 video.

(https://en.wikibooks.org/wiki/FFMPEG_An_Intermediate_Guide/image_sequence#Making_a_video_from_an_Image_Sequence)

Play around with your model. What happens if you introduce uncertainty about the car's brakes?